In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split

import keras.utils.np_utils as kutils

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,GRU,SimpleR
from keras.layers.advanced_activations import PReLU

from random import *

Загружаем таблицы банков

In [ ]:
banks = {}
for fname in sorted(os.listdir('data2')):
    banks[int(fname[:-4])] = pd.read_csv('data2/'+fname,encoding='UTF-8')

Отбираем в массивы номера лицензий закрывших банков и поныне работающих

In [ ]:
work = []
close = []
for i in banks.keys():
    if banks[i]['y7'][len(banks[i])-1]==-1:
        work.append(i)
        banks[i]['y8'] = 1+2*banks[i]['y7']
    else:
        close.append(i)
        banks[i]['y8'] = 1 - banks[i]['y7']
        banks[i]['y7'] -= banks[i]['y6']
        banks[i]['y6'] -= banks[i]['y5']
        banks[i]['y5'] -= banks[i]['y4']
        banks[i]['y4'] -= banks[i]['y3']
        banks[i]['y3'] -= banks[i]['y2']
        banks[i]['y2'] -= banks[i]['y1']
        banks[i]['y1'] -= banks[i]['y0']

Убираем банки, для которых нет наблюдений со всеми известными целевыми переменными

In [ ]:
short = []
for i in work:
    if len(banks[i])<36:
        banks.pop(i)
        short.append(i)
    else:
        banks[i] = banks[i][:-35]
for i in short:
    work.remove(i)

Забираем целевые переменные в отдельные таблицы

In [ ]:
razmetka = {}
for i in banks.keys():
    razmetka[i] = banks[i][['y'+str(j) for j in range(9)]]
    for k in ['y'+str(j) for j in range(9)]:
        del banks[i][k]
    del banks[i]['year']
    del banks[i]['month']

Разбиваем на тренировочную и тестовую выборку

In [ ]:
work_train, work_test = train_test_split(work,test_size=0.2,random_state=12)
close_train, close_test = train_test_split(close,test_size=0.2,random_state=15)

Аккуратно все группируем

In [ ]:
train = {}
test = {}
train_y = {}
test_y = {}
for i in banks.keys():
    if (i in work_train) or (i in close_train):
        train[i] = banks[i]
        train_y[i] = razmetka[i]
    else:
        test[i] = banks[i]
        test_y[i] = razmetka[i]

Вычисляем матожидание каждого признака и его дисперсию

In [ ]:
E = {}
D = {}
N = 0
for i in banks[1].columns:
    E[i] = 0
    D[i] = 0
for i in train.keys():
    for j in banks[1].columns:
        E[j] += np.sum(banks[i][j])
        D[j] += np.sum(banks[i][j]**2)
    N += len(banks[i])
for i in banks[1].columns:
    E[i] = E[i]/N
    D[i] = D[i]/N - E[i]**2
    D[i] = D[i]**0.5

Стандартизируем распределения признаков

In [ ]:
for i in train.keys():
    for j in banks[1].columns:
        train[i][j] = (train[i][j]-E[j])/D[j]
for i in test.keys():
    for j in banks[1].columns:
        test[i][j] = (test[i][j]-E[j])/D[j]

Переводим таблицы в numpy массивы

In [ ]:
for i in train.keys():
    train[i] = train[i].values.reshape(train[i].values.shape[0],1,train[i].values.shape[1])
for i in test.keys():
    test[i] = test[i].values.reshape(test[i].values.shape[0],1,test[i].values.shape[1])

Приводим целевую переменню в формат, пригодны для keras

In [ ]:
for i in train.keys():
    train_y[i] = train_y[i]['y1']+2*train_y[i]['y2']+3*train_y[i]['y3']+4*train_y[i]['y4']+5*train_y[i]['y5']+6*train_y[i]['y6']+7*train_y[i]['y7']+8*train_y[i]['y8']
    train_y[i] = kutils.to_categorical(train_y[i])
for i in test.keys():
    test_y[i] = test_y[i]['y1']+2*test_y[i]['y2']+3*test_y[i]['y3']+4*test_y[i]['y4']+5*test_y[i]['y5']+6*test_y[i]['y6']+7*test_y[i]['y7']+8*test_y[i]['y8']
    test_y[i] = kutils.to_categorical(test_y[i])

Форматируем, чтобы для каждого банка было одно число целевых дамми-переменных

In [ ]:
for i in train.keys():
    if train_y[i].shape[1]<9:
        train_y[i] = np.hstack((train_y[i],np.zeros((train_y[i].shape[0],9-train_y[i].shape[1]))))
for i in test.keys():
    if test_y[i].shape[1]<9:
        test_y[i] = np.hstack((test_y[i],np.zeros((test_y[i].shape[0],9-test_y[i].shape[1]))))

Число признаков и классов

In [ ]:
feat_num = train[1003].shape[2]
class_num = 9

Определяем F1-меру и более лояльную F1-меру

In [ ]:
def fmera(pred,orig):
    pres = 0
    rec = 0
    for i in range(9):
        if sum((pred==i))!=0:
            pres += sum((pred==i) & (orig==i)) / sum((pred==i))
        rec += sum((pred==i) & (orig==i)) / sum((orig==i))
    pres = pres/9
    rec = rec/9
    return 2*pres*rec/(pres+rec)
def fmera_mode(pred,orig):
    pres = 0
    rec = 0
    for i in range(9):
        if sum((pred==i))!=0:
            pres += sum((pred==i) & ((orig==i) | (orig==i+1) | (orig==i-1))) / sum((pred==i))
        rec += sum(((pred==i) | (pred==i-1) | (pred==i+1)) & (orig==i)) / sum((orig==i))
    pres = pres/9
    rec = rec/9
    return 2*pres*rec/(pres+rec)

Назначаем веса классам

In [ ]:
indexes = list(train.keys())
clsw = {8:0.1,7:1,6:1,5:1,4:1,3:1,2:1,1:1,0:1}

Строим stateful LSTM модель

In [ ]:
model = Sequential()
model.add(LSTM(120,batch_input_shape=(1, 1, feat_num), stateful=True, dropout_U=0.25, dropout_W=0.25))
model.add(Dropout(0.25))
model.add(Dense(class_num, activation="softmax", init='glorot_uniform'))
model.compile(loss="categorical_crossentropy", optimizer='adagrad')
model.summary()

Обучаем модель, выводя метрики на тестовой выборке, а также на тестовой выборке, давая модели полгода на обучение для каждого банка

In [ ]:
for i in range(60):
    for j in train.keys():
        model.fit(train[j],train_y[j], nb_epoch=1, batch_size=1,
                  shuffle=False, verbose=0,class_weight=clsw)
        model.reset_states()
    model.save_weights('models/model'+str(i))
    t = 0
    for j in test.keys():
        if t==0:
            pred = model.predict_classes(test[j],batch_size=1,verbose=False)
            model.reset_states()
            original = test_y[j][:,1]+2*test_y[j][:,2]+3*test_y[j][:,3]+4*test_y[j][:,4]+5*test_y[j][:,5]+6*test_y[j][:,6]+7*test_y[j][:,7]+8*test_y[j][:,8]
            pred_mod = pred[6:]
            original_mod = original[6:]
            t=1
            continue
        now = model.predict_classes(test[j],batch_size=1,verbose=False)
        model.reset_states()
        pred = np.hstack((pred,now))
        original = np.hstack((original,test_y[j][:,1]+2*test_y[j][:,2]+3*test_y[j][:,3]+4*test_y[j][:,4]+5*test_y[j][:,5]+6*test_y[j][:,6]+7*test_y[j][:,7]+8*test_y[j][:,8]))
        if len(now)>6:
            pred_mod = np.hstack((pred_mod,now[6:]))
            original_mod = np.hstack((original_mod,(test_y[j][:,1]+2*test_y[j][:,2]+3*test_y[j][:,3]+4*test_y[j][:,4]+5*test_y[j][:,5]+6*test_y[j][:,6]+7*test_y[j][:,7]+8*test_y[j][:,8])[6:]))
    print((str(i)+'):').ljust(7),str(np.mean(abs(original-pred)))[:6].ljust(9),str(sum(pred==original)/len(pred))[:6].ljust(9),\
         str(fmera(pred,original))[:6].ljust(9),str(fmera_mode(pred,original))[:6].ljust(9))
    print((str(i)+'):').ljust(7),str(np.mean(abs(original_mod-pred_mod)))[:6].ljust(9),str(sum(pred_mod==original_mod)/len(pred_mod))[:6].ljust(9),\
         str(fmera(pred_mod,original_mod))[:6].ljust(9),str(fmera_mode(pred_mod,original_mod))[:6].ljust(9),'\n')

Лучший результат, которого удалось достичь - это чистая F1-мера на тестовой выборке в районе 0.35, лояльная F1-мера на тестовой выборке в районе 0.65. В целом опыт интересный и полезный, но тяжело делать подобные проекты в одиночку и за бесплатно. Простора для улучшения результата очень много, однако, он требует времени и сил, но является более рутинной работой.